# DoF Estimator Testing Notebook

This notebook tests and evaluates the degrees of freedom (DoF) estimator implementation using simulated data matching the Monte Carlo simulation setup.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import project modules
from fspb.simulation.model_simulation import (
    simulate_from_model,
    generate_default_time_grid,
)
from fspb.bands.covariance import dof_estimate
from fspb.bands.linear_model import ConcurrentLinearModel
from fspb.types import CovarianceType
from fspb.config import LENGTH_SCALE

# Set up plotting style
plt.style.use("default")
np.random.seed(42)

## 1. Define Test Parameters

We'll test the DoF estimator across different scenarios matching your simulation setup.

In [2]:
# Test parameters - matching the simulation scenarios
test_scenarios = {
    "n_samples": [30, 100],
    "true_dof": [5, 15],
    "covariance_types": [CovarianceType.STATIONARY, CovarianceType.NON_STATIONARY],
}

# Number of replications for each scenario
n_replications = 100

# Generate time grid (same as in simulation)
time_grid = generate_default_time_grid()
print(
    f"Time grid: {len(time_grid)} points from {time_grid[0]:.2f} to {time_grid[-1]:.2f}"
)
print(
    f"Test scenarios: {len(test_scenarios['n_samples']) * len(test_scenarios['true_dof']) * len(test_scenarios['covariance_types'])} combinations"
)
print(
    f"Total simulations: {n_replications * len(test_scenarios['n_samples']) * len(test_scenarios['true_dof']) * len(test_scenarios['covariance_types'])}"
)

Time grid: 101 points from 0.00 to 1.00
Test scenarios: 8 combinations
Total simulations: 800


## 2. Helper Functions for DoF Estimation Testing

In [3]:
def generate_residuals(
    n_samples: int,
    true_dof: int,
    covariance_type: CovarianceType,
    rng: np.random.Generator,
) -> np.ndarray:
    """
    Generate residuals by simulating data and fitting the linear model.

    Returns:
        residuals: Shape (n_samples, n_time_points)
    """
    # Simulate data
    sim_data = simulate_from_model(
        n_samples=n_samples,
        time_grid=time_grid,
        dof=true_dof,
        covariance_type=covariance_type,
        length_scale=LENGTH_SCALE,
        rng=rng,
    )

    # Fit the model to get residuals
    model = ConcurrentLinearModel()
    model.fit(sim_data.x, sim_data.y)

    # Calculate residuals
    y_pred = model.predict(sim_data.x)
    residuals = sim_data.y - y_pred

    return residuals


def test_dof_estimator_single(
    n_samples: int, true_dof: int, covariance_type: CovarianceType, n_reps: int = 100
) -> dict:
    """
    Test DoF estimator for a single scenario with multiple replications.
    """
    rng = np.random.default_rng(42)  # Fixed seed for reproducibility

    estimated_dofs = []

    for rep in range(n_reps):
        # Generate residuals
        residuals = generate_residuals(n_samples, true_dof, covariance_type, rng)

        # Estimate DoF directly
        estimated_dof = dof_estimate(residuals)
        estimated_dofs.append(estimated_dof)

    estimated_dofs = np.array(estimated_dofs)

    # Remove any infinite or NaN values
    valid_estimates = estimated_dofs[np.isfinite(estimated_dofs)]

    if len(valid_estimates) == 0:
        raise ValueError("No valid estimates found")

    return {
        "n_samples": n_samples,
        "covariance_type": covariance_type.value,
        "true_dof": true_dof,
        "mean_estimate": np.mean(valid_estimates),
        "std_estimate": np.std(valid_estimates),
        "n_valid_estimates": len(valid_estimates),
    }

## 3. Run DoF Estimator Tests

Now let's test the DoF estimator across all scenarios.

In [4]:
# Run comprehensive DoF estimator testing
results = []

total_scenarios = (
    len(test_scenarios["n_samples"])
    * len(test_scenarios["true_dof"])
    * len(test_scenarios["covariance_types"])
)
scenario_count = 0

print(f"Running DoF estimator tests across {total_scenarios} scenarios...")

for n_samples in test_scenarios["n_samples"]:
    for true_dof in test_scenarios["true_dof"]:
        for covariance_type in test_scenarios["covariance_types"]:
            scenario_count += 1

            print(
                f"\rScenario {scenario_count}/{total_scenarios}: n={n_samples}, dof={true_dof}, cov={covariance_type.value}",
                end="",
            )

            result = test_dof_estimator_single(
                n_samples=n_samples,
                true_dof=true_dof,
                covariance_type=covariance_type,
                n_reps=n_replications,
            )

            results.append(result)

print("\nTesting completed!")

# Convert to DataFrame for easier analysis
results_df = pd.DataFrame(
    [{k: v for k, v in r.items() if k != "estimates"} for r in results]
)
print(f"\nResults shape: {results_df.shape}")
results_df

Running DoF estimator tests across 8 scenarios...
Scenario 2/8: n=30, dof=5, cov=non_stationary

/home/tim/sciebo-thinky/fspb/src/fspb/simulation/model_simulation.py:200: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  z = rng.multivariate_normal(


Scenario 8/8: n=100, dof=15, cov=non_stationary
Testing completed!

Results shape: (8, 6)


,n_samples,covariance_type,true_dof,mean_estimate,std_estimate,n_valid_estimates
0,30,stationary,5,12.358789,10.491349,100
1,30,non_stationary,5,6.876454,4.004040,100
2,30,stationary,15,19.920391,13.036374,100
3,30,non_stationary,15,12.298393,9.067814,100
4,100,stationary,5,6.614364,3.034688,100
5,100,non_stationary,5,5.332273,0.887578,100
6,100,stationary,15,15.231938,9.801465,100
7,100,non_stationary,15,9.762254,4.518714,100
